<h1> Identifying spam in popular YouTube videos comments </h1>
<p1>TODO: Short Description of the data and the approach.</p1>
<p2>TODO: Description of the code in all cells </p2>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import metrics
import zipfile
COMMENTS_FT = 'comments_words'
ZIP_FILE = 'data/YouTube-Spam-Collection-v1.zip'
DIR_TO_EXTRACT = 'data/'
EMBED_DIMENSION = 50

#Extract the Data
zip_ref = zipfile.ZipFile(ZIP_FILE, 'r')
zip_ref.extractall(DIR_TO_EXTRACT)
zip_ref.close()

/Users/i346047/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/i346047/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
comments_df_list = []
comments_file = ['data/Youtube01-Psy.csv','data/Youtube02-KatyPerry.csv','data/Youtube03-LMFAO.csv',
                'data/Youtube04-Eminem.csv','data/Youtube05-Shakira.csv']
for f in comments_file:
    df = pd.read_csv(f,header=0)
    comments_df_list.append(df)
comments_df = pd.concat(comments_df_list)
comments_df = comments_df.sample(frac=1.0)
print(comments_df.shape)
comments_df.head(5)

(1956, 5)


,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
66,z12yzzqoozqieho4x04ci12qbrzhyzyyvjg,iSmartKid,2015-05-28T03:27:18.079000,2015 but Im still listening to this!﻿,0
21,z13tyhj4drvcxzllt222tvwp3qi1e34ju04,Inga Lomidze,NaN,like please﻿,1
32,z120tv4aborbjtpuo22ecxvx2rqmsrhck,Paige Lebrun,2015-05-26T23:25:22.172000,"LMFAO - Party Rock Anthem ft. Lauren Bennett, ...",0
158,z12tyfpzkznlghgqz04cjpsp3ymuil05t3g,sheerwill,2015-05-18T13:01:00.402000,I like this song<br />﻿,0
112,z12mzfsy0mypfvcy404ccdvqmyq5gzqazow0k,Nancy “Aya” Lo,2015-05-21T19:16:44.104000,I came here because of Vanoss.﻿,0


In [3]:
comments_df.columns

Index(['COMMENT_ID', 'AUTHOR', 'DATE', 'CONTENT', 'CLASS'], dtype='object')

In [4]:
average_comments_size = int(sum([len(c) for c in comments_df.CONTENT])/comments_df.shape[0])
print(average_comments_size)

94


In [5]:
voc_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(average_comments_size)
X_transform = voc_processor.fit_transform(comments_df.CONTENT)
X_transform = np.array(list(X_transform))
y = comments_df.CLASS.values
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_transform, 
                                    y, test_size=0.2, random_state=42)
n_words = len(voc_processor.vocabulary_)

In [6]:
voc_dict = voc_processor.vocabulary_._mapping
sorted_vocab = sorted(voc_dict.items(), key = lambda x : x[1])
f = open('/tmp/meta.tsv', 'w')
for val in sorted_vocab:
    f.write(str(val[0]) + "\n")
f.close()

In [7]:
def get_estimator_spec(input_logits, out_lb, train_predict_m):
    preds_cls = tf.argmax(input_logits, 1)
    if train_predict_m == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
        mode=train_predict_m,
        predictions={
            'pred_class': preds_cls,
            'pred_prob': tf.nn.softmax(input_logits)
        })
    tr_l = tf.losses.sparse_softmax_cross_entropy(labels=out_lb, logits=input_logits)
    if train_predict_m == tf.estimator.ModeKeys.TRAIN:
        adm_opt = tf.train.AdamOptimizer(learning_rate=0.01)
        tr_op = adm_opt.minimize(tr_l, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(train_predict_m, loss=tr_l, train_op=tr_op)
    eval_metric_ops = {'accuracy': tf.metrics.accuracy(labels=out_lb, predictions=preds_cls)}
    return tf.estimator.EstimatorSpec(train_predict_m, loss=tr_l, train_op=tr_op)

In [8]:
def rnn_model_fn(features, labels, mode):
    comments_wd_vec = tf.contrib.layers.embed_sequence(
      features[COMMENTS_FT], vocab_size=n_words, embed_dim=EMBED_DIMENSION)
    comments_word_list = tf.unstack(comments_wd_vec, axis=1)
    
    rnn_cell = tf.nn.rnn_cell.GRUCell(average_comments_size)
    
    _, comments_encoding = tf.nn.static_rnn(rnn_cell, comments_word_list, dtype=tf.float32)
    
    logits = tf.layers.dense(inputs=comments_encoding, units=2, activation=None)
    
    return get_estimator_spec(input_logits=logits, out_lb=labels, train_predict_m=mode)

In [9]:
run_config = tf.contrib.learn.RunConfig()
run_config = run_config.replace(model_dir='/tmp/models/',save_summary_steps=10,log_step_count_steps=10)
classifier = tf.estimator.Estimator(model_fn=rnn_model_fn,config=run_config)

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x114558da0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_secs': 600, '_log_step_count_steps': 10, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/models/'}


In [10]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={COMMENTS_FT: X_train},
      y=y_train,
      batch_size=128,
      num_epochs=None,
      shuffle=True)
classifier.train(input_fn=train_input_fn, steps=200)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/models/model.ckpt.
INFO:tensorflow:loss = 0.6945967, step = 1
INFO:tensorflow:global_step/sec: 3.30829
INFO:tensorflow:global_step/sec: 11.6651
INFO:tensorflow:global_step/sec: 11.3617
INFO:tensorflow:global_step/sec: 11.6671
INFO:tensorflow:global_step/sec: 11.6703
INFO:tensorflow:global_step/sec: 11.5843
INFO:tensorflow:global_step/sec: 11.6649
INFO:tensorflow:global_step/sec: 11.6708
INFO:tensorflow:global_step/sec: 11.3528
INFO:tensorflow:global_step/sec: 11.7895
INFO:tensorflow:loss = 0.31859905, step = 101 (10.782 sec)
INFO:tensorflow:global_step/sec: 11.4417
INFO:tensorflow:global_step/sec: 11.7339
INFO:tensorflow:global_step/sec: 11.7294
INFO:tensorflow:global_step/sec: 11.7192
INFO:tensorflow:global_step/sec: 11.381
INFO:tensorflow:global_step/sec: 11.6948
INFO:tensorflow:global_step/sec: 11.7341
INFO:tensorflow:global_step/sec: 11.6495
INFO:tensorflow:global_step/sec: 11.6934
INFO:t

In [11]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={COMMENTS_FT: X_test},
      y=y_test,
      num_epochs=1,
      shuffle=False)
preds = classifier.predict(input_fn=test_input_fn)
y_predicted = np.array(list(p['pred_class'] for p in preds))
y_predicted = y_predicted.reshape(np.array(y_test).shape)

acc = metrics.accuracy_score(y_test, y_predicted)
print('Accuracy: {0:f}'.format(acc))

INFO:tensorflow:Restoring parameters from /tmp/models/model.ckpt-200
Accuracy: 0.905612
